# <span style="color:green">Formation South Green 2022</span> - Structural Variants Detection by using short and long reads 

# __DAY 2 : SNP calling__

Created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)

## __1. Preparing the working environment__ 

### First create a dedicated folder to work 

In [17]:
# go to work directory
cd /home/jovyan/work/
ls

BAM_ILL      MAPPING-ONT  SV_DATA               training_SV_teaching_old
MAPPING-ILL  SNIFFLES     training_SV_teaching  VCF


### We are going to use only one clones to check is all is ok before run the whole of samples! 

In [2]:
## declare variables
i=10
REF_DIR="/home/jovyan/work/SV_DATA/REF/"
REF="/home/jovyan/work/SV_DATA/REF/reference.fasta"
ONT="/home/jovyan/work/SV_DATA/LONG_READS/Clone$i.fastq.gz"
ILL_R1="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R1.fastq.gz"
ILL_R2="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R2.fastq.gz"
BAM="/home/jovyan/work/MAPPING-ILL/dirClone10/Clone$i.SORTED.bam"

In [19]:
# go to SR mapping results
cd /home/jovyan/work/MAPPING-ILL/dirClone${i}

In [20]:
ls /home/jovyan/work/MAPPING-ILL/dirClone${i}

Clone10.bam                     Clone10.SORTED.bam.flagstat
Clone10.bam.flagstat            Clone10.SORTED.MD.bam
Clone10.flagstats               Clone10.SORTED.MD.bam.bai
Clone10.mappedpaired.flagstats  Clone10.SORTED.MD.bam.flagstat
Clone10.SORTED.bam              duplicates.10.metrics
Clone10.SORTED.bam.bai


### Mark duplications

In [ ]:
echo -e "\nMarkDuplicates in Clone$i";
gatk MarkDuplicates -I $BAM -M duplicates.$i.metrics -O Clone$i.SORTED.MD.bam;
samtools index Clone$i.SORTED.MD.bam;

In [ ]:
# check MD bam
ls -l

### Indexing reference with GATK

In [ ]:
cd $REF_DIR
samtools faidx $REF
gatk CreateSequenceDictionary -R $REF

### SNP Calling for a Clone

In [ ]:
# go to bam repertory
cd /home/jovyan/work/MAPPING-ILL/dirClone${i}
# change BAM file for MD one
BAM="/home/jovyan/work/MAPPING-ILL/dirClone10/Clone$i.SORTED.MD.bam"
# lauch GATK HaplotypeCaller
echo -e "\nCalling Clone$i";
gatk --java-options "-Xmx4g" HaplotypeCaller --native-pair-hmm-threads 4 -I Clone$i.SORTED.MD.bam -O Clone$i.g.vcf -R $REF -ERC GVCF;

In [ ]:
head Clone$i.g.vcf

### Error produced here is normal ! RG is important in mapping step  !! 

Yes, you have to relaunch mapping with all samples ! BUT ...

Don't worry, bam files with correct `-R "@RG\tID:Clone${i}\tSM:Clone${i}"` parameter in bwa-mem2 command are available for downloading.

In [ ]:
cd /home/jovyan/work/
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" wget https://itrop.ird.fr/sv-training/BAM_ILL.tar.gz
tar zxvf BAM_ILL.tar.gz
BAM_ILL="/home/jovyan/work/BAM_ILL"
rm BAM_ILL.tar.gz
ls $BAM_ILL

## Calling all samples on one raw VCF with correct BAM files

In [3]:
BAM_ILL="/home/jovyan/work/BAM_ILL"
ls $BAM_ILL

all_stat.csv                 Clone2.SORTED.bam.flagstat
BAM_ILL.tar.gz               Clone2.SORTED.MD.bam
Clone10.SORTED.bam           Clone2.SORTED.MD.bam.bai
Clone10.SORTED.bam.flagstat  Clone3.SORTED.bam
Clone10.SORTED.MD.bam        Clone3.SORTED.bam.flagstat
Clone10.SORTED.MD.bam.bai    Clone3.SORTED.MD.bam
Clone11.SORTED.bam           Clone3.SORTED.MD.bam.bai
Clone11.SORTED.bam.flagstat  Clone4.SORTED.bam
Clone11.SORTED.MD.bam        Clone4.SORTED.bam.flagstat
Clone11.SORTED.MD.bam.bai    Clone4.SORTED.MD.bam
Clone12.SORTED.bam           Clone4.SORTED.MD.bam.bai
Clone12.SORTED.bam.flagstat  Clone5.SORTED.bam
Clone12.SORTED.MD.bam        Clone5.SORTED.bam.flagstat
Clone12.SORTED.MD.bam.bai    Clone5.SORTED.MD.bam
Clone13.SORTED.bam           Clone5.SORTED.MD.bam.bai
Clone13.SORTED.bam.flagstat  Clone6.SORTED.bam
Clone13.SORTED.MD.bam        Clone6.SORTED.bam.flagstat
Clone13.SORTED.MD.bam.bai    Clone6.SORTED.MD.bam
Clone14.SORTED.bam           Clone6.SORTED.MD.bam.bai
Clone14.SORT

In [4]:
# create a work directory to SNP calling results
mkdir -p /home/jovyan/work/VCF
VCF_DIR="/home/jovyan/work/VCF"

### Markduplicates

In [ ]:
cd $BAM_ILL
for i in {1..20}
    do
        samtools index Clone$i.SORTED.bam;
        echo -e "\nMarkDuplicates in Clone$i";
        gatk MarkDuplicates -I Clone$i.SORTED.bam -M duplicates.$i.metrics -O Clone$i.SORTED.MD.bam;
        samtools index Clone$i.SORTED.MD.bam;
    done

### HaplotypeCaller

In [5]:
cd $BAM_ILL
for i in {10..20}
    do
        # lauch GATK HaplotypeCaller
        echo -e "\nCalling Clone$i";
        gatk --java-options "-Xmx4g" HaplotypeCaller --native-pair-hmm-threads 4 -I Clone$i.SORTED.MD.bam -O $VCF_DIR/Clone$i.g.vcf -R $REF -ERC GVCF
    done


Calling Clone10
Using GATK jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -Xmx4g -jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar HaplotypeCaller --native-pair-hmm-threads 4 -I Clone10.SORTED.MD.bam -O /home/jovyan/work/VCF/Clone10.g.vcf -R /home/jovyan/work/SV_DATA/REF/reference.fasta -ERC GVCF
16:48:38.853 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar!/com/intel/gkl/native/libgkl_compression.so
16:48:39.448 INFO  HaplotypeCaller - ------------------------------------------------------------
16:48:39.448 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.2.6.1
16:48:39.448 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
1

### CombineGVCF

In [6]:
BAM_ILL="/home/jovyan/work/BAM_ILL"
VCF_DIR="/home/jovyan/work/VCF"
REF="/home/jovyan/work/SV_DATA/REF/reference.fasta"

# change of work directory
cd $VCF_DIR
# Loop to inflate the --variant option
OPTION=""
for i in {1..20}
do
    OPTION="${OPTION} --variant Clone${i}.g.vcf"
done
echo $OPTION
# GATK
gatk CombineGVCFs -R $REF $OPTION -O rawSNP.vcf

--variant Clone1.g.vcf --variant Clone2.g.vcf --variant Clone3.g.vcf --variant Clone4.g.vcf --variant Clone5.g.vcf --variant Clone6.g.vcf --variant Clone7.g.vcf --variant Clone8.g.vcf --variant Clone9.g.vcf --variant Clone10.g.vcf --variant Clone11.g.vcf --variant Clone12.g.vcf --variant Clone13.g.vcf --variant Clone14.g.vcf --variant Clone15.g.vcf --variant Clone16.g.vcf --variant Clone17.g.vcf --variant Clone18.g.vcf --variant Clone19.g.vcf --variant Clone20.g.vcf
Using GATK jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar CombineGVCFs -R /home/jovyan/work/SV_DATA/REF/reference.fasta --variant Clone1.g.vcf --variant Clone2.g.vcf --variant Clone3.g.vcf --variant Clone4.g.vcf --variant Clone5.g.vcf --variant Clone6.g.vcf --varian

### Have a Look to it combineGVCF

In [7]:
head -n 1000 rawSNP.vcf | tail

Reference	901	.	G	T,<NON_REF>	.	.	DP=708;ExcessHet=0.00;RAW_MQandDP=306000,85	GT:AD:DP:GQ:MIN_DP:PGT:PID:PL:PS:SB	./.:.:27:75:25:.:.:0,75,977,75,977,977	./.:.:44:99:43:.:.:0,114,1710,114,1710,1710	./.:.:34:93:34:.:.:0,93,1395,93,1395,1395	./.:.:49:99:47:.:.:0,117,1755,117,1755,1755	./.:.:43:99:41:.:.:0,117,1755,117,1755,1755	./.:.:42:99:41:.:.:0,117,1755,117,1755,1755	./.:.:35:99:34:.:.:0,102,1382,102,1382,1382	./.:.:21:60:20:.:.:0,60,803,60,803,803	./.:.:20:60:20:.:.:0,60,801,60,801,801	./.:.:44:99:41:.:.:0,117,1755,117,1755,1755	./.:.:0:0:0:.:.:0,0,0,0,0,0	.|.:0,43,0:43:99:.:0|1:901_G_T:1908,129,0,1908,129,1908:901:0,0,22,21	./.:.:46:99:45:.:.:0,120,1800,120,1800,1800	./.:.:48:99:46:.:.:0,117,1755,117,1755,1755	.|.:0,42,0:42:99:.:0|1:809_A_T:1890,126,0,1890,126,1890:809:0,0,21,21	./.:.:37:99:36:.:.:0,99,1485,99,1485,1485	./.:.:46:99:43:.:.:0,112,1755,112,1755,1755	./.:.:31:90:31:.:.:0,90,1227,90,1227,1227	./.:.:45:99:43:.:.:0,108,1620,108,1620,1620	./.:.:34:90:33:.:.:0,90,1350,90,135

### Compute the Genotypes

In [8]:
gatk --java-options "-Xmx4g" GenotypeGVCFs -R $REF -V rawSNP.vcf -O output.vcf

Using GATK jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -Xmx4g -jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar GenotypeGVCFs -R /home/jovyan/work/SV_DATA/REF/reference.fasta -V rawSNP.vcf -O output.vcf
22:28:35.187 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar!/com/intel/gkl/native/libgkl_compression.so
22:28:35.823 INFO  GenotypeGVCFs - ------------------------------------------------------------
22:28:35.824 INFO  GenotypeGVCFs - The Genome Analysis Toolkit (GATK) v4.2.6.1
22:28:35.824 INFO  GenotypeGVCFs - For support and documentation go to https://software.broadinstitute.org/gatk/
22:28:35.830 INFO  GenotypeGVCFs - Initializing engine
22:28:36.020 INFO  FeatureManager - Using cod

In [9]:
head output.vcf

##fileformat=VCFv4.2
##ALT=<ID=NON_REF,Description="Represents any possible alternative allele not already represented at this location by REF and ALT">
##FILTER=<ID=LowQual,Description="Low quality">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=MIN_DP,Number=1,Type=Integer,Description="Minimum DP observed within the GVCF block">
##FORMAT=<ID=PGT,Number=1,Type=String,Description="Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another; will always be heterozygous and is not intended to describe called alleles">
##FORMAT=<ID=PID,Number=1,Type=String,Description="Physical phasing ID information

### Compute the SNP density along the chromosomes

In [10]:
echo -e "Reference\t1000000\n" > genome.txt
bedtools genomecov -bga -split -i output.vcf -g genome.txt > density.csv

In [11]:
head density.csv

Reference	0	29	0
Reference	29	30	1
Reference	30	35	0
Reference	35	36	1
Reference	36	37	0
Reference	37	39	1
Reference	39	41	0
Reference	41	42	1
Reference	42	44	0
Reference	44	45	1


### Count the number of variants with `bcftools stat`

In [12]:
bcftools stats output.vcf > output.vcf.stats

In [13]:
head -n 100 output.vcf.stats

# This file was produced by bcftools stats (1.10.2+htslib-1.10.2-3) and can be plotted using plot-vcfstats.
# The command line was:	bcftools stats  output.vcf
#
# Definition of sets:
# ID	[2]id	[3]tab-separated file names
ID	0	output.vcf
# SN, Summary numbers:
#   number of records   .. number of data rows in the VCF
#   number of no-ALTs   .. reference-only sites, ALT is either "." or identical to REF
#   number of SNPs      .. number of rows with a SNP
#   number of MNPs      .. number of rows with a MNP, such as CC>TT
#   number of indels    .. number of rows with an indel
#   number of others    .. number of rows with other type, for example a symbolic allele or
#                          a complex substitution, such as ACT>TCGA
#   number of multiallelic sites     .. number of rows with multiple alternate alleles
#   number of multiallelic SNP sites .. number of rows with multiple alternate alleles, all SNPs
# 
#   Note that rows containing multiple types will be counted multiple 